# 填充和步幅

## 填充

- 给定$(32\times32)$输入图像
- 应用$5\times5$大小的卷积核
  - 第一层得到输出大小$28\times28$
  - 第七层得到输出大小$4\times4$
- 更大的卷积核可以更快地减小输出大小
  - 形状从$n_h \times n_w$减少到$(n_h -k_h +1)\times(n_w -k_w +1)$

- 填充：在输入的四周加入额外的行/列，以达到更大的输入
- 填充$p_h$行和$p_w$列,输出的形状为$$(n_h -k_h +p_h +1)\times(n_w -k_w +p_w +1)$$
- 通常取：$p_h = k_h -1,\quad p_w = k_w -1$
  - 当$k_h$为奇数：上下两侧填充$p_h /2$
  - 当$k_h$为偶数：在上侧填充$\lceil p_h /2\rceil$，在下侧填充$\lfloor p_h /2\rfloor$

## 步幅
- 填充减小的输出大小与层数线性相关
  - 给定输入大小$224\times 224$，在使用$5\times5$卷积核的情况下，需要55层将输出降低到$4\times4$
  - 需要大量的计算测i能得到较小输出
- 步幅是指行/列的滑动步长
- 给定高度$s_h$和宽度$s_w$的步幅，输出形状是$$\lfloor (n_h -k_h +p_h +s_h)/s_h \rfloor\times\lfloor(n_w -k_w +p_w +s_w)\rfloor $$
- 如果$p_h =k_h -1,\quad p_w =k_w -1$则：$$\lfloor (n_h  +s_h -1)/s_h \rfloor\times\lfloor(n_w  +s_w -1)\rfloor $$
- 如果输入高度和宽度可以被步幅整除$(n_h /s_h)\times(n_w /s_w)$

## 总结
- 填充和步幅都是卷积层的超参数
- 填充在输入周围添加额外的行/列，来控制输出形状的减少量
- 步幅是每次滑动核窗口时的行/列的步长，可以成倍的减少输出的形状

# 代码实现

在所有侧边填充一个像素

In [1]:
import torch
from torch import nn


# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

填充不同的高度和宽度

In [2]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

将高度和宽度的步幅设置为2

In [3]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

稍微复杂的例子

In [4]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])